In [1]:
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
from meteostat import Point, Daily
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
import numpy as np
import warnings

In [2]:
df_cargo = pd.read_csv('db-cargo-delays-2016-deutschland.csv')
df_cargo.head()

,BST_NR_8,Land,PROD_DATUM,Zugfahrten,Verspaetungsminuten,dayofweek,weekofyear,month
0,80197079,DEUTSCHLAND,2016-03-16,39,49,3,11,3
1,80160168,DEUTSCHLAND,2016-01-11,12,0,1,2,1
2,80105122,DEUTSCHLAND,2016-04-30,16,2,6,17,4
3,80190793,DEUTSCHLAND,2016-07-31,14,0,7,30,7
4,80231464,DEUTSCHLAND,2016-04-05,10,2,2,14,4


In [12]:
df_geo = pd.read_csv('geo-bahnstellen-export.csv', delimiter=';')
df_geo = df_geo.rename(columns={'BST8': 'BST_NR_8', 'LAT': 'latitude', 'LON': 'longitude'})
df_geo.head()

,BST_NR_8,BST_NAME,LAND,latitude,longitude
0,11759,NaN,NaN,NaN,NaN
1,28100,SWIEBODZIN,NaN,52.247296,15.533572
2,254581,NaN,NaN,NaN,NaN
3,824532,NaN,NaN,NaN,NaN
4,999995,NaN,NaN,NaN,NaN


In [13]:
len(df_cargo), len(df_geo)

(771621, 51432)

In [15]:
df = df_cargo.merge(df_geo[['BST_NR_8', 'latitude', 'longitude']])
del df_cargo
del df_geo
df.head()

,BST_NR_8,Land,PROD_DATUM,Zugfahrten,Verspaetungsminuten,dayofweek,weekofyear,month,latitude,longitude
0,80197079,DEUTSCHLAND,2016-03-16,39,49,3,11,3,49.42045,7.613668
1,80197079,DEUTSCHLAND,2016-10-11,38,74,2,41,10,49.42045,7.613668
2,80197079,DEUTSCHLAND,2016-08-29,27,33,1,35,8,49.42045,7.613668
3,80197079,DEUTSCHLAND,2016-02-25,53,85,4,8,2,49.42045,7.613668
4,80197079,DEUTSCHLAND,2016-05-28,39,38,6,21,5,49.42045,7.613668
